<a href="https://colab.research.google.com/github/Hanbin-git/Dacon_cacer/blob/main/2025_06_23(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -o "/content/drive/MyDrive/open_1.zip" -d "/content/open_1"


Archive:  /content/drive/MyDrive/open_1.zip
  inflating: /content/open_1/sample_submission.csv  
  inflating: /content/open_1/test.csv  
  inflating: /content/open_1/train.csv  


In [3]:
import os

def get_path(filename):
    return "/content/open_1/" + filename


In [4]:
# SMOTE 설치
!pip install -U imbalanced-learn
!pip install lightgbm optuna
!pip install -q lightgbm catboost xgboost



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.3 MB/s eta 0:00:00


In [5]:
!pip install --upgrade lightgbm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 46.8 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully uninstalled lightgbm-4.5.0


In [6]:
import lightgbm as lgb
print(lgb.__version__)

4.6.0


In [7]:
#  Optuna 기반 LGBMClassifier 메타모델 튜닝 + Stacking 전체 코드
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.ensemble import StackingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 경로 함수
def get_path(filename):
    return "/content/open_1/" + filename  # 환경에 맞게 경로 수정

# ✅ 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리 함수
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

X, encoders = preprocess(X)
X_test, _ = preprocess(X_test, encoders)

# ✅ Optuna 목적 함수 정의
def objective(trial):
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "num_leaves": trial.suggest_int("num_leaves", 15, 100),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 5.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 5.0),
        "random_state": 42,
        "n_estimators": 100
    }

    meta_model = LGBMClassifier(**params)
    base_models = [
        ('lgbm', LGBMClassifier(random_state=42)),
        ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
        ('cat', CatBoostClassifier(verbose=0, random_state=42))
    ]

    stack_model = StackingClassifier(
        estimators=base_models,
        final_estimator=meta_model,
        cv=5,
        stack_method="predict_proba",
        n_jobs=-1
    )

    oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
    return f1_score(y, oof_preds > 0.2)  # 고정 threshold

# ✅ Optuna 최적화 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

print("Best trial:", study.best_trial.params)

# ✅ 최적 파라미터로 재학습
best_params = study.best_trial.params
meta_model = LGBMClassifier(**best_params)

base_models = [
    ('lgbm', LGBMClassifier(random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
    ('cat', CatBoostClassifier(verbose=0, random_state=42))
]

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    stack_method="predict_proba",
    n_jobs=-1
)

# ✅ Threshold 재탐색
oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# ✅ 전체 학습 후 예측 및 제출
stack_model.fit(X, y)
test_preds = stack_model.predict_proba(X_test)[:, 1]
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("submission_optuna_stacking_lgb.csv", index=False)
print("✅ 제출 완료 → submission_optuna_stacking_lgb.csv 저장됨")



[I 2025-06-23 00:18:43,319] A new study created in memory with name: no-name-e3f80385-c486-445c-93a6-9b20cab0b3d0


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003066 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008038 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


...

[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007930 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003104 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[W 2025-06-23 02:13:44,157] Trial 13 failed with parameters: {'learning_rate': 0.10519496948165993, 'max_depth': 5, 'num_leaves': 35, 'min_child_samples': 99, 'subsample': 0.7229084675583938, 'colsample_bytree': 0.926144916114375, 'reg_alpha': 3.9120471161443016, 'reg_lambda': 1.0511169856334595} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-7-1237435294.py", line 74, in objective
    oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/loca

KeyboardInterrupt: 

In [ ]:
from google.colab import files
files.download("submission_optuna_stacking_lgb.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# [전략 2] 전체 스택(Base + Meta) 구조 통합 Optuna 튜닝 전체 코드
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.ensemble import StackingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 경로 함수
def get_path(filename):
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리 함수
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

X, encoders = preprocess(X)
X_test, _ = preprocess(X_test, encoders)

# ✅ Optuna 전체 구조 튜닝 목적 함수 정의
def objective(trial):
    # Base LGBM
    lgbm_params = {
        "learning_rate": trial.suggest_float("lgbm_lr", 0.01, 0.2),
        "max_depth": trial.suggest_int("lgbm_max_depth", 3, 7),
        "num_leaves": trial.suggest_int("lgbm_leaves", 15, 63),
        "random_state": 42,
        "n_estimators": 100
    }
    lgb_model = LGBMClassifier(**lgbm_params)

    # Base XGB
    xgb_params = {
        "learning_rate": trial.suggest_float("xgb_lr", 0.01, 0.2),
        "max_depth": trial.suggest_int("xgb_max_depth", 3, 7),
        "subsample": trial.suggest_float("xgb_subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("xgb_colsample", 0.6, 1.0),
        "eval_metric": "logloss",
        "use_label_encoder": False,
        "random_state": 42,
        "n_estimators": 100
    }
    xgb_model = XGBClassifier(**xgb_params)

    # Base CatBoost
    cat_params = {
        "learning_rate": trial.suggest_float("cat_lr", 0.01, 0.2),
        "depth": trial.suggest_int("cat_depth", 3, 7),
        "random_state": 42,
        "verbose": 0,
        "n_estimators": 100
    }
    cat_model = CatBoostClassifier(**cat_params)

    # Meta LGBM
    meta_params = {
        "learning_rate": trial.suggest_float("meta_lr", 0.01, 0.3),
        "max_depth": trial.suggest_int("meta_max_depth", 3, 6),
        "num_leaves": trial.suggest_int("meta_leaves", 15, 63),
        "random_state": 42,
        "n_estimators": 100
    }
    meta_model = LGBMClassifier(**meta_params)

    # 전체 스택 모델 구성
    base_models = [
        ("lgbm", lgb_model),
        ("xgb", xgb_model),
        ("cat", cat_model)
    ]
    stack_model = StackingClassifier(
        estimators=base_models,
        final_estimator=meta_model,
        cv=5,
        stack_method="predict_proba",
        n_jobs=-1
    )

    oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
    return f1_score(y, oof_preds > 0.2)

# ✅ Optuna 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

print("✅ Best trial:", study.best_trial.params)

# ✅ 최적 파라미터로 학습 및 예측
params = study.best_trial.params

# base 모델
lgb_model = LGBMClassifier(
    learning_rate=params["lgbm_lr"],
    max_depth=params["lgbm_max_depth"],
    num_leaves=params["lgbm_leaves"],
    random_state=42, n_estimators=100
)
xgb_model = XGBClassifier(
    learning_rate=params["xgb_lr"],
    max_depth=params["xgb_max_depth"],
    subsample=params["xgb_subsample"],
    colsample_bytree=params["xgb_colsample"],
    eval_metric="logloss", use_label_encoder=False,
    random_state=42, n_estimators=100
)
cat_model = CatBoostClassifier(
    learning_rate=params["cat_lr"],
    depth=params["cat_depth"],
    random_state=42, verbose=0, n_estimators=100
)

meta_model = LGBMClassifier(
    learning_rate=params["meta_lr"],
    max_depth=params["meta_max_depth"],
    num_leaves=params["meta_leaves"],
    random_state=42, n_estimators=100
)

# 전체 스택 모델
stack_model = StackingClassifier(
    estimators=[
        ("lgbm", lgb_model),
        ("xgb", xgb_model),
        ("cat", cat_model)
    ],
    final_estimator=meta_model,
    cv=5,
    stack_method="predict_proba",
    n_jobs=-1
)

# Threshold 최적화
oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"📌 Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# 최종 학습 및 제출
stack_model.fit(X, y)
test_preds = stack_model.predict_proba(X_test)[:, 1]
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("submission_stacking_full_optuna.csv", index=False)
print("✅ 최종 제출 완료 → submission_stacking_full_optuna.csv")



In [ ]:
from google.colab import files
files.download("submission_stacking_full_optuna.csv")


In [ ]:
# ✅ 기본 라이브러리
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import optuna
import warnings
warnings.filterwarnings("ignore")

# ✅ 데이터 경로
path = "D:/traffic-project/"
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
submission = pd.read_csv(path + "sample_submission.csv")

# ✅ Target 분리
X = train.drop(columns=["ID", "Cancer"])
y = train["Cancer"]
X_test = test.drop(columns=["ID"])

# ✅ 결측값 처리 및 인코딩
cat_cols = X.select_dtypes(include="object").columns
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))
    encoders[col] = le

imputer = SimpleImputer(strategy="mean")
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# ✅ 튜닝 대상 전체 모델 구조
def get_stacked_model(trial):
    # Base learners
    xgb = XGBClassifier(
        n_estimators=trial.suggest_int("xgb_n", 100, 300),
        max_depth=trial.suggest_int("xgb_max", 3, 8),
        learning_rate=trial.suggest_float("xgb_lr", 0.01, 0.3),
        use_label_encoder=False,
        eval_metric="logloss",
        verbosity=0,
        random_state=42
    )
    cat = CatBoostClassifier(
        iterations=trial.suggest_int("cat_n", 100, 300),
        learning_rate=trial.suggest_float("cat_lr", 0.01, 0.3),
        depth=trial.suggest_int("cat_depth", 4, 8),
        verbose=0,
        random_state=42
    )
    # Meta learner
    lgbm = LGBMClassifier(
        n_estimators=trial.suggest_int("lgbm_n", 100, 300),
        learning_rate=trial.suggest_float("lgbm_lr", 0.01, 0.3),
        max_depth=trial.suggest_int("lgbm_max", 3, 8),
        random_state=42
    )

    model = StackingClassifier(
        estimators=[("xgb", xgb), ("cat", cat)],
        final_estimator=lgbm,
        cv=3,
        passthrough=True,
        n_jobs=-1
    )
    return model

# ✅ Optuna 튜닝 목적함수
def objective(trial):
    model = get_stacked_model(trial)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    preds = []
    true = []

    for train_idx, val_idx in skf.split(X, y):
        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        pred = model.predict(X.iloc[val_idx])
        preds.extend(pred)
        true.extend(y.iloc[val_idx])

    return f1_score(true, preds)

# ✅ Optuna 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# ✅ 최적 모델 학습
best_model = get_stacked_model(study.best_trial)
best_model.fit(X, y)

# ✅ 예측 및 제출
preds = best_model.predict_proba(X_test)[:, 1]
submission["Cancer"] = (preds > 0.20).astype(int)  # threshold=0.20 적용
submission.to_csv(path + "submission_stacked_optuna.csv", index=False)
print("✅ 최종 제출 저장 완료:", path + "submission_stacked_optuna.csv")


In [ ]:
# ✅ 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import optuna
import warnings
warnings.filterwarnings("ignore")

# ✅ 데이터 경로
path = "D:/traffic-project/"
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
submission = pd.read_csv(path + "sample_submission.csv")

# ✅ Target 분리
X = train.drop(columns=["ID", "Cancer"])
y = train["Cancer"]
X_test = test.drop(columns=["ID"])

# ✅ 인코딩 + 결측치 처리
cat_cols = X.select_dtypes(include="object").columns
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))
    encoders[col] = le

imputer = SimpleImputer(strategy="mean")
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# ✅ 모델 구성 함수
def get_stacked_model(trial):
    xgb = XGBClassifier(
        n_estimators=trial.suggest_int("xgb_n", 100, 300),
        max_depth=trial.suggest_int("xgb_max", 3, 8),
        learning_rate=trial.suggest_float("xgb_lr", 0.01, 0.3),
        use_label_encoder=False,
        eval_metric="logloss",
        verbosity=0,
        random_state=42
    )
    cat = CatBoostClassifier(
        iterations=trial.suggest_int("cat_n", 100, 300),
        learning_rate=trial.suggest_float("cat_lr", 0.01, 0.3),
        depth=trial.suggest_int("cat_depth", 4, 8),
        verbose=0,
        random_state=42
    )
    lgbm = LGBMClassifier(
        n_estimators=trial.suggest_int("lgbm_n", 100, 300),
        learning_rate=trial.suggest_float("lgbm_lr", 0.01, 0.3),
        max_depth=trial.suggest_int("lgbm_max", 3, 8),
        random_state=42
    )
    model = StackingClassifier(
        estimators=[("xgb", xgb), ("cat", cat)],
        final_estimator=lgbm,
        cv=3,
        passthrough=True,
        n_jobs=-1
    )
    return model

# ✅ Optuna 목적 함수
def objective(trial):
    model = get_stacked_model(trial)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    preds, true = [], []

    for train_idx, val_idx in skf.split(X, y):
        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        pred = model.predict(X.iloc[val_idx])
        preds.extend(pred)
        true.extend(y.iloc[val_idx])

    return f1_score(true, preds)

# ✅ Optuna 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# ✅ 최적 모델 학습
best_model = get_stacked_model(study.best_trial)
best_model.fit(X, y)

# ✅ Threshold 최적화
train_probs = best_model.predict_proba(X)[:, 1]
thresholds = np.arange(0.05, 0.95, 0.01)
f1s = [f1_score(y, train_probs > t) for t in thresholds]
best_idx = np.argmax(f1s)
best_thr = thresholds[best_idx]
best_f1 = f1s[best_idx]

print(f"✅ Best Threshold: {best_thr:.2f} | Best F1 Score: {best_f1:.5f}")

# ✅ Test 예측 및 저장
test_probs = best_model.predict_proba(X_test)[:, 1]
submission["Cancer"] = (test_probs > best_thr).astype(int)

save_path = path + f"submission_thresh_optuna_{best_thr:.2f}.csv"
submission.to_csv(save_path, index=False)
print(f"📁 최종 제출 저장 완료: {save_path}")


In [ ]:
#  [전략 3] SHAP 기반 Feature 조합 + 메타 튜닝 (Feature set을 변수화)
# 단순히 모델만 튜닝하지 않고, 사용할 feature 개수도 변수로 설정

# top_n_features를 튜닝에 포함 → top_10, top_20, top_30 실험을 자동화

# 장점: 모델과 데이터 구조를 함께 최적화함

# 예: Optuna trial에서 n_features = trial.suggest_categorical('n_feats', [10, 20, 30])

# ✅ [전략 4] Threshold + Meta 파라미터 동시 최적화
# meta model 파라미터 + best_threshold를 동시에 탐색

# 기존처럼 F1 score 기준으로 threshold 튜닝하는 대신, Optuna가 threshold까지 함께 학습

# 예:

# python
# 복사
# 편집
# threshold = trial.suggest_float('threshold', 0.1, 0.9)
# f1 = f1_score(y_val, (probs > threshold).astype(int))
# ✅ [전략 5] LGBM Classifier의 early_stopping을 활용한 내장 검증 기반 튜닝
# Optuna가 내부적으로 학습을 줄이도록 구성 → early_stopping_rounds를 활용하여 자동 검증 조기종료

# fit() 내부에서 eval_set=(X_val, y_val)로 조기 종료 조건 주면 빠르게 수렴 가능

# fit() 안에서 LGBM에 eval_metric='auc' or 'binary_logloss' 등 넣어 fine control

# ✅ [전략 6] Optuna + StratifiedKFold 기반 Robust 튜닝
# train_test_split이 아닌 StratifiedKFold로 validation split 하여 안정적인 성능 확보

# 특히 불균형 데이터에 적합

# Optuna에서 objective 함수 내부에 KFold 반복 적용하여 평균 F1 반환

# 조금 느리지만 더 신뢰도 높은 튜닝 전략

